In [1]:
# Importing libraries
import pyarrow.parquet as pq
import pandas as pd

In [2]:
# Function to process the parquet files and return a single dataframe
def process_data(file_paths):
    dataframes = []
    for file_path in file_paths:
        table = pq.read_table(file_path)# Reading the parquet file as a table
        df = table.to_pandas()# Converting the table to a pandas dataframe
        dataframes.append(df)# Appending the dataframe to the list
    
    # Standardizing column names based on the first dataframe
    col_names_std = dataframes[0].columns.tolist()
    for df in dataframes:
        df.columns = col_names_std
    
    # Printing size of the dataframes
    for i, df in enumerate(dataframes):
        print(f"Shape of dataframe {i+1}: ", df.shape)
    print("Total Size of all dataframes: ", sum([df.shape[0] for df in dataframes]))

    # Concatenating the dataframes
    combined_df = pd.concat(dataframes, ignore_index=True)
    print("Shape of combined dataframe: ", combined_df.shape)
    
    return combined_df

In [3]:
# Reading the parquet files
file_paths = ['D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\Monthly\y2023\yellow_tripdata_2023-01.parquet',
              'D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\Monthly\y2023\yellow_tripdata_2023-02.parquet',
              'D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\Monthly\y2023\yellow_tripdata_2023-03.parquet',
              'D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\Monthly\y2023\yellow_tripdata_2023-04.parquet',
              'D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\Monthly\y2023\yellow_tripdata_2023-05.parquet',
              'D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\Monthly\y2023\yellow_tripdata_2023-06.parquet',
              'D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\Monthly\y2023\yellow_tripdata_2023-07.parquet',
              ]
yellow_tripdata_2023 = process_data(file_paths)

Shape of dataframe 1:  (3066766, 19)
Shape of dataframe 2:  (2913955, 19)
Shape of dataframe 3:  (3403766, 19)
Shape of dataframe 4:  (3288250, 19)
Shape of dataframe 5:  (3513649, 19)
Shape of dataframe 6:  (3307234, 19)
Shape of dataframe 7:  (2907108, 19)
Total Size of all dataframes:  22400728
Shape of combined dataframe:  (22400728, 19)


In [4]:
# Checking for null values
nan_in_each_column = yellow_tripdata_2023.isna().sum()
print(nan_in_each_column)

VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          613638
trip_distance                 0
RatecodeID               613638
store_and_fwd_flag       613638
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge     613638
airport_fee              613638
dtype: int64


In [5]:
# Filling null values
yellow_tripdata_2023['congestion_surcharge'] = yellow_tripdata_2023['congestion_surcharge'].fillna(0)
yellow_tripdata_2023['airport_fee'] = yellow_tripdata_2023['airport_fee'].fillna(0)
yellow_tripdata_2023['passenger_count'] = yellow_tripdata_2023['passenger_count'].fillna(0)
yellow_tripdata_2023['RatecodeID'] = yellow_tripdata_2023['RatecodeID'].fillna(1.0)
yellow_tripdata_2023['store_and_fwd_flag'] = yellow_tripdata_2023['store_and_fwd_flag'].fillna('N')

In [6]:
# Checking for null values again
nan_in_each_column = yellow_tripdata_2023.isna().sum()
print(nan_in_each_column)

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
airport_fee              0
dtype: int64


In [7]:
# Checking datatypes
yellow_tripdata_2023.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [8]:
# Converting datatypes
yellow_tripdata_2023['passenger_count'] = yellow_tripdata_2023['passenger_count'].astype('int64')
yellow_tripdata_2023['RatecodeID'] = yellow_tripdata_2023['RatecodeID'].astype('int64')
yellow_tripdata_2023['store_and_fwd_flag'] = yellow_tripdata_2023['store_and_fwd_flag'].astype(str)

In [9]:
# Checking datatypes again
yellow_tripdata_2023.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                   int64
trip_distance                   float64
RatecodeID                        int64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [10]:
# Checking for invalid values
mask1 = ~yellow_tripdata_2023['VendorID'].isin([1, 2]) # Check 'VendorID' column contains values other than 1 and 2
mask2 = ~yellow_tripdata_2023['RatecodeID'].isin([1, 2, 3, 4, 5, 6]) # Check 'RatecodeID' column contains values other than 1, 2, 3, 4, 5, and 6
mask3 = ~yellow_tripdata_2023['store_and_fwd_flag'].isin(['Y', 'N']) # Check if 'store_and_fwd_flag' column contains values other than 'Y' and 'N'
mask4 = ~yellow_tripdata_2023['payment_type'].isin([1, 2, 3, 4, 5, 6]) # Check if 'payment_type' column contains values other than 1, 2, 3, 4, 5, and 6
print(mask1.sum())
print(mask2.sum())
print(mask3.sum())
print(mask4.sum())

5820
110397
0
613638


In [11]:
# Dropping invalid values for VendorID
yellow_tripdata_2023 = yellow_tripdata_2023.drop(yellow_tripdata_2023[mask1].index)

In [12]:
# Replace the values in RatecodeID column with 1 if they are invalid
yellow_tripdata_2023.loc[mask2, 'RatecodeID'] = 1

In [13]:
# Replace the values in payment_type column with 5 if they are invalid
yellow_tripdata_2023.loc[mask4, 'payment_type'] = 5

In [14]:
# Checking for invalid values again
mask1 = ~yellow_tripdata_2023['VendorID'].isin([1, 2]) # Check 'VendorID' column contains values other than 1 and 2
mask2 = ~yellow_tripdata_2023['RatecodeID'].isin([1, 2, 3, 4, 5, 6]) # Check 'RatecodeID' column contains values other than 1, 2, 3, 4, 5, and 6
mask3 = ~yellow_tripdata_2023['store_and_fwd_flag'].isin(['Y', 'N']) # Check if 'store_and_fwd_flag' column contains values other than 'Y' and 'N'
mask4 = ~yellow_tripdata_2023['payment_type'].isin([1, 2, 3, 4, 5, 6]) # Check if 'payment_type' column contains values other than 1, 2, 3, 4, 5, and 6
print(mask1.sum())
print(mask2.sum())
print(mask3.sum())
print(mask4.sum())

0
0
0
0


In [15]:
# Total no. of rows after dropping invalid values
yellow_tripdata_2023.shape[0]

22394908

In [16]:
# Shorting the dataframe removing some random data to reduce the size
print("Size before: ",yellow_tripdata_2023.shape[0])
n = 10594908
if n < len(yellow_tripdata_2023):
    drop_indices = yellow_tripdata_2023.sample(n, random_state=42).index
    yellow_tripdata_2023 = yellow_tripdata_2023.drop(drop_indices)
else:
    print(f"Your DataFrame has only {len(yellow_tripdata_2023)} rows. Can't drop {n} rows.")
print("Size After: ",yellow_tripdata_2023.shape[0])

Size before:  22394908
Size After:  11800000


In [17]:
# Dropping duplicates
print("Size before: ",yellow_tripdata_2023.shape[0])
yellow_tripdata_2023 = yellow_tripdata_2023.drop_duplicates().reset_index(drop=True)
print("Size After: ",yellow_tripdata_2023.shape[0])

Size before:  11800000
Size After:  11800000


In [18]:
# Viewing info of the dataframe
yellow_tripdata_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11800000 entries, 0 to 11799999
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        int64         
 4   trip_distance          float64       
 5   RatecodeID             int64         
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee         

In [19]:
# Viewing description of the dataframe
yellow_tripdata_2023.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,1.180000e+07,11800000,11800000,1.180000e+07,1.180000e+07,1.180000e+07,1.180000e+07,1.180000e+07,1.180000e+07,1.180000e+07,1.180000e+07,1.180000e+07,1.180000e+07,1.180000e+07,1.180000e+07,1.180000e+07,1.180000e+07,1.180000e+07
mean,1.732656e+00,2023-04-17 02:51:55.251289,2023-04-17 03:08:55.164616,1.331335e+00,4.043167e+00,1.071378e+00,1.652791e+02,1.639313e+02,1.327568e+00,1.919760e+01,1.584485e+00,4.867596e-01,3.491887e+00,5.719915e-01,9.809874e-01,2.808658e+01,2.206842e+00,1.262431e-01
min,1.000000e+00,2001-01-01 00:06:49,2001-01-01 00:16:31,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-8.700000e+02,-7.500000e+00,-5.000000e-01,-4.110000e+02,-6.155000e+01,-1.000000e+00,-9.005000e+02,-2.500000e+00,-1.750000e+00
25%,1.000000e+00,2023-02-25 05:47:12.750000,2023-02-25 06:11:00.750000,1.000000e+00,1.070000e+00,1.000000e+00,1.320000e+02,1.130000e+02,1.000000e+00,9.300000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.575000e+01,2.500000e+00,0.000000e+00
50%,2.000000e+00,2023-04-18 07:31:11.500000,2023-04-18 07:46:14.500000,1.000000e+00,1.800000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.350000e+01,1.000000e+00,5.000000e-01,2.800000e+00,0.000000e+00,1.000000e+00,2.070000e+01,2.500000e+00,0.000000e+00
75%,2.000000e+00,2023-06-06 14:33:18.500000,2023-06-06 14:52:22.250000,1.000000e+00,3.440000e+00,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,2.120000e+01,2.500000e+00,5.000000e-01,4.340000e+00,0.000000e+00,1.000000e+00,3.020000e+01,2.500000e+00,0.000000e+00
max,2.000000e+00,2023-10-16 15:09:40,2023-10-16 15:13:42,9.000000e+00,2.589281e+05,6.000000e+00,2.650000e+02,2.650000e+02,5.000000e+00,6.300900e+03,6.733000e+01,4.000000e+00,9.843000e+02,6.655600e+02,1.000000e+00,6.304900e+03,2.500000e+00,1.750000e+00
std,4.425734e-01,NaN,NaN,9.118776e-01,2.104379e+02,3.810813e-01,6.401432e+01,6.992440e+01,7.935916e-01,1.877961e+01,1.838638e+00,1.043347e-01,4.043806e+00,2.155710e+00,1.914199e-01,2.338409e+01,8.586649e-01,4.343392e-01


In [20]:
# Viewing the dataframe after preprocessing
yellow_tripdata_2023.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1,0.97,1,N,161,141,2,9.3,1.0,0.5,0.00,0.0,1.0,14.30,2.5,0.0
1,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1,1.43,1,N,107,79,1,11.4,1.0,0.5,3.28,0.0,1.0,19.68,2.5,0.0
2,2,2023-01-01 00:50:34,2023-01-01 01:02:52,1,1.84,1,N,161,137,1,12.8,1.0,0.5,10.00,0.0,1.0,27.80,2.5,0.0
3,2,2023-01-01 00:27:12,2023-01-01 00:49:56,1,11.70,1,N,142,200,1,45.7,1.0,0.5,10.74,3.0,1.0,64.44,2.5,0.0
4,2,2023-01-01 00:21:44,2023-01-01 00:36:40,1,2.95,1,N,164,236,1,17.7,1.0,0.5,5.68,0.0,1.0,28.38,2.5,0.0
5,1,2023-01-01 00:43:37,2023-01-01 01:17:18,4,7.30,1,N,79,264,1,33.8,3.5,0.5,7.75,0.0,1.0,46.55,2.5,0.0
6,2,2023-01-01 00:33:53,2023-01-01 00:49:15,1,2.95,1,N,33,61,1,17.7,1.0,0.5,4.04,0.0,1.0,24.24,0.0,0.0
7,2,2023-01-01 00:13:04,2023-01-01 00:22:10,1,1.52,1,N,79,186,1,10.0,1.0,0.5,1.25,0.0,1.0,16.25,2.5,0.0
8,2,2023-01-01 00:45:11,2023-01-01 01:07:39,1,2.23,1,N,90,48,1,19.8,1.0,0.5,4.96,0.0,1.0,29.76,2.5,0.0
9,1,2023-01-01 00:03:36,2023-01-01 00:09:36,3,1.20,1,N,237,239,2,8.6,3.5,0.5,0.00,0.0,1.0,13.60,2.5,0.0


In [21]:
# Viewing the dataframe after preprocessing
yellow_tripdata_2023.tail(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
11799990,1,2023-07-31 23:29:00,2023-07-31 23:43:06,0,0.00,1,N,79,100,5,15.00,1.0,0.5,4.00,0.00,1.0,26.00,0.0,0.0
11799991,2,2023-07-31 23:19:45,2023-07-31 23:36:15,0,5.19,1,N,249,238,5,23.15,0.0,0.5,4.07,0.00,1.0,31.22,0.0,0.0
11799992,2,2023-07-31 23:35:12,2023-08-01 00:00:39,0,10.70,1,N,255,74,5,39.27,0.0,0.5,0.00,6.55,1.0,47.32,0.0,0.0
11799993,2,2023-07-31 23:11:00,2023-07-31 23:49:00,0,7.46,1,N,143,256,5,35.38,0.0,0.5,0.00,0.00,1.0,39.38,0.0,0.0
11799994,2,2023-07-31 23:16:00,2023-07-31 23:29:00,0,2.85,1,N,68,162,5,15.78,0.0,0.5,3.96,0.00,1.0,23.74,0.0,0.0
11799995,2,2023-07-31 23:55:10,2023-07-31 23:58:40,0,1.00,1,N,61,188,5,14.44,0.0,0.5,0.00,0.00,1.0,15.94,0.0,0.0
11799996,2,2023-07-31 23:18:11,2023-07-31 23:18:23,0,0.00,1,N,106,106,5,21.58,0.0,0.5,0.00,0.00,1.0,23.08,0.0,0.0
11799997,2,2023-07-31 23:29:31,2023-07-31 23:51:17,0,4.97,1,N,238,243,5,22.12,0.0,0.5,0.00,0.00,1.0,23.62,0.0,0.0
11799998,2,2023-07-31 23:29:00,2023-07-31 23:46:00,0,3.24,1,N,65,62,5,9.54,0.0,0.5,3.00,0.00,1.0,14.04,0.0,0.0
11799999,2,2023-07-31 23:34:13,2023-07-31 23:48:51,0,2.08,1,N,162,143,5,14.33,0.0,0.5,0.00,0.00,1.0,18.33,0.0,0.0


In [22]:
# Writing the dataframe to a parquet file
yellow_tripdata_2023.to_parquet('D:\TLC-NYC-Data-Analytics-End-To-End\env\Data\yellow_tripdata_2023.parquet.gzip', compression='gzip')